<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [6]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("BIS_cbrate.dta")
stata_df.head()

,period,period_start_day,value,obs_conf,obs_status,frequency,dataset_code,dataset_name,freq,ref_area,indexed_at,provider_code,series_code,series_name,series_num
0,1993-04,1993-04-01,0.63,F,A,monthly,WS_CBPOL,Central bank policy rates,M,AR,2025-06-13T15:24:25.093Z,BIS,M.AR,Central bank policy rates - Argentina - Month...,Master
1,1993-05,1993-05-01,0.11,F,A,monthly,WS_CBPOL,Central bank policy rates,M,AR,2025-06-13T15:24:25.093Z,BIS,M.AR,Central bank policy rates - Argentina - Month...,Master
2,1993-06,1993-06-01,0.53,F,A,monthly,WS_CBPOL,Central bank policy rates,M,AR,2025-06-13T15:24:25.093Z,BIS,M.AR,Central bank policy rates - Argentina - Month...,Master
3,1993-07,1993-07-01,0.36,F,A,monthly,WS_CBPOL,Central bank policy rates,M,AR,2025-06-13T15:24:25.093Z,BIS,M.AR,Central bank policy rates - Argentina - Month...,Master
4,1993-08,1993-08-01,0.25,F,A,monthly,WS_CBPOL,Central bank policy rates,M,AR,2025-06-13T15:24:25.093Z,BIS,M.AR,Central bank policy rates - Argentina - Month...,Master


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [7]:
python_df = pd.read_pickle("BIS_cbrate.pkl")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,FREQ,REF_AREA,Frequency,Reference area
0,monthly,BIS,WS_CBPOL,Central bank policy rates,M.AR,Central bank policy rates - Argentina - Month...,1993-04,1993-04-01,0.63,0.63,M,AR,Monthly,Argentina
1,monthly,BIS,WS_CBPOL,Central bank policy rates,M.AR,Central bank policy rates - Argentina - Month...,1993-05,1993-05-01,0.11,0.11,M,AR,Monthly,Argentina
2,monthly,BIS,WS_CBPOL,Central bank policy rates,M.AR,Central bank policy rates - Argentina - Month...,1993-06,1993-06-01,0.53,0.53,M,AR,Monthly,Argentina
3,monthly,BIS,WS_CBPOL,Central bank policy rates,M.AR,Central bank policy rates - Argentina - Month...,1993-07,1993-07-01,0.36,0.36,M,AR,Monthly,Argentina
4,monthly,BIS,WS_CBPOL,Central bank policy rates,M.AR,Central bank policy rates - Argentina - Month...,1993-08,1993-08-01,0.25,0.25,M,AR,Monthly,Argentina


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [8]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [9]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [10]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [11]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True